In [14]:
import pandas as pd

request = pd.read_excel('./sample_data.xlsx', sheet_name="request")
impression = pd.read_excel('./sample_data.xlsx', sheet_name="impression")
feedback = pd.read_excel('./sample_data.xlsx', sheet_name="feedback")

feedback

,request_id,track_id,clicked,skipped_10s,listened_30s,liked,listened_sec
0,REQ_0001,TRK_111,1,0,1,1,30
1,REQ_0001,TRK_005,1,0,1,0,31
2,REQ_0001,TRK_028,1,0,0,0,10
3,REQ_0001,TRK_040,0,0,0,0,0
4,REQ_0001,TRK_041,0,0,0,0,0
...,...,...,...,...,...,...,...
245,REQ_0050,TRK_054,1,0,1,0,106
246,REQ_0050,TRK_001,1,0,0,0,26
247,REQ_0050,TRK_008,1,0,0,0,27
248,REQ_0050,TRK_094,1,0,0,0,28


relevance bucket 계산해서 feedback 데이터에 추가

In [25]:
import numpy as np

feedback["relevance"] = 0

conditions = [
    feedback["liked"] == 1,
    feedback["listened_30s"] == 1,
    feedback["clicked"] == 1
]

choices = [3,2,1]

feedback["relevance"] = np.select(conditions, choices, 0)

feedback

,request_id,track_id,clicked,skipped_10s,listened_30s,liked,listened_sec,relevance
0,REQ_0001,TRK_111,1,0,1,1,30,3
1,REQ_0001,TRK_005,1,0,1,0,31,2
2,REQ_0001,TRK_028,1,0,0,0,10,1
3,REQ_0001,TRK_040,0,0,0,0,0,0
4,REQ_0001,TRK_041,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
245,REQ_0050,TRK_054,1,0,1,0,106,2
246,REQ_0050,TRK_001,1,0,0,0,26,1
247,REQ_0050,TRK_008,1,0,0,0,27,1
248,REQ_0050,TRK_094,1,0,0,0,28,1


In [26]:
# 학습 단위(track 하나) 테이블 만들기
df = (
    impression
    .merge(feedback, on=["request_id", "track_id"], how="left")
    .merge(request, on="request_id", how="left")
)

df

,policy_id,request_id,track_id,track_feature1,track_feature2,track_feature3,track_feature4,rank,clicked,skipped_10s,...,user_id,created_at,context_text,mood_vector,dominant_mood,mood_level_max,mood_entropy,situation_tag,place_tag,weather_tag
0,POL_RULE_V1,REQ_0001,TRK_111,0.735,0.470,0.675,0.145,1,1,0,...,U_001,2025-01-01 08:12:00,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.42,commuting,home,cloudy
1,POL_RULE_V1,REQ_0001,TRK_005,0.752,0.504,0.760,0.264,2,1,0,...,U_001,2025-01-01 08:12:00,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.42,commuting,home,cloudy
2,POL_RULE_V1,REQ_0001,TRK_028,0.785,0.570,0.925,0.495,3,1,0,...,U_001,2025-01-01 08:12:00,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.42,commuting,home,cloudy
3,POL_RULE_V1,REQ_0001,TRK_040,0.741,0.482,0.705,0.187,4,0,0,...,U_001,2025-01-01 08:12:00,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.42,commuting,home,cloudy
4,POL_RULE_V1,REQ_0001,TRK_041,0.748,0.496,0.740,0.236,5,0,0,...,U_001,2025-01-01 08:12:00,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.42,commuting,home,cloudy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,POL_RULE_V2,REQ_0050,TRK_054,0.775,0.550,0.875,0.425,1,1,0,...,U_001,2025-01-23 22:15:00,하루 마무리하면서 차분해지고 싶어,"[0,2,0,1,0,2,2]",joy,2,1.09,sleeping,home,night
246,POL_RULE_V2,REQ_0050,TRK_001,0.724,0.448,0.620,0.068,2,1,0,...,U_001,2025-01-23 22:15:00,하루 마무리하면서 차분해지고 싶어,"[0,2,0,1,0,2,2]",joy,2,1.09,sleeping,home,night
247,POL_RULE_V2,REQ_0050,TRK_008,0.773,0.546,0.865,0.411,3,1,0,...,U_001,2025-01-23 22:15:00,하루 마무리하면서 차분해지고 싶어,"[0,2,0,1,0,2,2]",joy,2,1.09,sleeping,home,night
248,POL_RULE_V2,REQ_0050,TRK_094,0.799,0.598,0.995,0.593,4,1,0,...,U_001,2025-01-23 22:15:00,하루 마무리하면서 차분해지고 싶어,"[0,2,0,1,0,2,2]",joy,2,1.09,sleeping,home,night


In [30]:
import ast

#Shannon Entropy
def mood_entropy(mood_vector):
    x = np.array(mood_vector, dtype=float)
    if x.sum() == 0:
        return 0.0
    p = x / x.sum()
    p = p[p>0]
    return -np.sum(p*np.log(p))

df["mood_vector_parsed"] = df["mood_vector"].apply(ast.literal_eval) 
df["mood_entropy"] = df["mood_vector_parsed"].apply(mood_entropy)

df["mood_entropy"]

0      1.689245
1      1.689245
2      1.689245
3      1.689245
4      1.689245
         ...   
245    1.351784
246    1.351784
247    1.351784
248    1.351784
249    1.351784
Name: mood_entropy, Length: 250, dtype: float64

In [31]:
df.to_excel("./sample_data_2.xlsx")

In [32]:
df.columns

Index(['policy_id', 'request_id', 'track_id', 'track_feature1',
       'track_feature2', 'track_feature3', 'track_feature4', 'rank', 'clicked',
       'skipped_10s', 'listened_30s', 'liked', 'listened_sec', 'relevance',
       'user_id', 'created_at', 'context_text', 'mood_vector', 'dominant_mood',
       'mood_level_max', 'mood_entropy', 'situation_tag', 'place_tag',
       'weather_tag', 'mood_vector_parsed'],
      dtype='object')

In [33]:
mood_cols = ["anger","sadness","pain","anxiety","shame","joy","love"]

mood_df = pd.DataFrame(df["mood_vector_parsed"].tolist(),
                       columns=[f"mood_{c}" for c in mood_cols])

mood_df

,mood_anger,mood_sadness,mood_pain,mood_anxiety,mood_shame,mood_joy,mood_love
0,1,7,3,4,1,2,1
1,1,7,3,4,1,2,1
2,1,7,3,4,1,2,1
3,1,7,3,4,1,2,1
4,1,7,3,4,1,2,1
...,...,...,...,...,...,...,...
245,0,2,0,1,0,2,2
246,0,2,0,1,0,2,2
247,0,2,0,1,0,2,2
248,0,2,0,1,0,2,2


In [34]:
dominant_mood = pd.get_dummies(df["dominant_mood"], 
                               columns=["dominant_mood"], 
                               prefix="dominant_mood")

situation_tag = pd.get_dummies(df["situation_tag"],
                               columns=["situation_tag"],
                               prefix="situation")

place_tag = pd.get_dummies(df["place_tag"],
                           columns=["place_tag"],
                           prefix="place")

weather_tag = pd.get_dummies(df["weather_tag"],
                             columns=["weather_tag"],
                             prefix="weather")

weather_tag

,weather_clear,weather_cloudy,weather_night,weather_rain
0,False,True,False,False
1,False,True,False,False
2,False,True,False,False
3,False,True,False,False
4,False,True,False,False
...,...,...,...,...
245,False,False,True,False
246,False,False,True,False
247,False,False,True,False
248,False,False,True,False


In [35]:
X = pd.concat([mood_df, dominant_mood, df["mood_level_max"], df["mood_entropy"],
                situation_tag, place_tag, weather_tag,
                df["track_feature1"],df["track_feature2"], df["track_feature3"], df["track_feature4"]], axis=1).astype(float)

X

,mood_anger,mood_sadness,mood_pain,mood_anxiety,mood_shame,mood_joy,mood_love,dominant_mood_anxiety,dominant_mood_joy,dominant_mood_love,...,place_outdoor,place_transport,weather_clear,weather_cloudy,weather_night,weather_rain,track_feature1,track_feature2,track_feature3,track_feature4
0,1.0,7.0,3.0,4.0,1.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.735,0.470,0.675,0.145
1,1.0,7.0,3.0,4.0,1.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.752,0.504,0.760,0.264
2,1.0,7.0,3.0,4.0,1.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.785,0.570,0.925,0.495
3,1.0,7.0,3.0,4.0,1.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.741,0.482,0.705,0.187
4,1.0,7.0,3.0,4.0,1.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.748,0.496,0.740,0.236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0.0,2.0,0.0,1.0,0.0,2.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.775,0.550,0.875,0.425
246,0.0,2.0,0.0,1.0,0.0,2.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.724,0.448,0.620,0.068
247,0.0,2.0,0.0,1.0,0.0,2.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.773,0.546,0.865,0.411
248,0.0,2.0,0.0,1.0,0.0,2.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.799,0.598,0.995,0.593


In [38]:
y = df["relevance"].astype(float)

y

0      3.0
1      2.0
2      1.0
3      0.0
4      0.0
      ... 
245    2.0
246    1.0
247    1.0
248    1.0
249    0.0
Name: relevance, Length: 250, dtype: float64

In [39]:
sort_cols = ["request_id", "rank"]

df = df.sort_values(sort_cols).reset_index(drop=True)
X = X.loc[df.index].reset_index(drop=True)
y = y.loc[df.index].reset_index(drop=True)

y

0      3.0
1      2.0
2      1.0
3      0.0
4      0.0
      ... 
245    2.0
246    1.0
247    1.0
248    1.0
249    0.0
Name: relevance, Length: 250, dtype: float64

In [40]:
req_ids = df["request_id"].unique()
rng = np.random.default_rng(33) 
rng.shuffle(req_ids)

In [41]:
n_train = int(len(req_ids)*0.8) 
train_reqs = set(req_ids[:n_train])
test_reqs = set(req_ids[n_train:])

train_mask = df["request_id"].isin(train_reqs)
test_mask = df["request_id"].isin(test_reqs)

X_train, y_train = X[train_mask], y[train_mask]
X_test, y_test = X[test_mask], y[test_mask]

df_test = df[test_mask].copy()

df_test

,policy_id,request_id,track_id,track_feature1,track_feature2,track_feature3,track_feature4,rank,clicked,skipped_10s,...,created_at,context_text,mood_vector,dominant_mood,mood_level_max,mood_entropy,situation_tag,place_tag,weather_tag,mood_vector_parsed
25,POL_HYBRID_V1,REQ_0006,TRK_111,0.735,0.470,0.675,0.145,1,1,0,...,2025-01-03 07:55:00,상쾌한 아침이야 기분 좋아,"[0,0,0,1,0,8,4]",joy,8,0.858741,morning,home,clear,"[0, 0, 0, 1, 0, 8, 4]"
26,POL_HYBRID_V1,REQ_0006,TRK_117,0.777,0.554,0.885,0.439,2,1,0,...,2025-01-03 07:55:00,상쾌한 아침이야 기분 좋아,"[0,0,0,1,0,8,4]",joy,8,0.858741,morning,home,clear,"[0, 0, 0, 1, 0, 8, 4]"
27,POL_HYBRID_V1,REQ_0006,TRK_120,0.734,0.468,0.670,0.138,3,1,0,...,2025-01-03 07:55:00,상쾌한 아침이야 기분 좋아,"[0,0,0,1,0,8,4]",joy,8,0.858741,morning,home,clear,"[0, 0, 0, 1, 0, 8, 4]"
28,POL_HYBRID_V1,REQ_0006,TRK_046,0.783,0.566,0.915,0.481,4,0,0,...,2025-01-03 07:55:00,상쾌한 아침이야 기분 좋아,"[0,0,0,1,0,8,4]",joy,8,0.858741,morning,home,clear,"[0, 0, 0, 1, 0, 8, 4]"
29,POL_HYBRID_V1,REQ_0006,TRK_083,0.786,0.572,0.930,0.502,5,0,0,...,2025-01-03 07:55:00,상쾌한 아침이야 기분 좋아,"[0,0,0,1,0,8,4]",joy,8,0.858741,morning,home,clear,"[0, 0, 0, 1, 0, 8, 4]"
35,POL_RULE_V2,REQ_0008,TRK_016,0.765,0.530,0.825,0.355,1,1,0,...,2025-01-04 10:05:00,카페에서 집중해서 공부 중이야,"[0,1,1,2,0,3,1]",joy,3,1.494175,studying,cafe,clear,"[0, 1, 1, 2, 0, 3, 1]"
36,POL_RULE_V2,REQ_0008,TRK_049,0.804,0.608,0.020,0.628,2,1,0,...,2025-01-04 10:05:00,카페에서 집중해서 공부 중이야,"[0,1,1,2,0,3,1]",joy,3,1.494175,studying,cafe,clear,"[0, 1, 1, 2, 0, 3, 1]"
37,POL_RULE_V2,REQ_0008,TRK_007,0.766,0.532,0.830,0.362,3,1,0,...,2025-01-04 10:05:00,카페에서 집중해서 공부 중이야,"[0,1,1,2,0,3,1]",joy,3,1.494175,studying,cafe,clear,"[0, 1, 1, 2, 0, 3, 1]"
38,POL_RULE_V2,REQ_0008,TRK_080,0.765,0.530,0.825,0.355,4,0,0,...,2025-01-04 10:05:00,카페에서 집중해서 공부 중이야,"[0,1,1,2,0,3,1]",joy,3,1.494175,studying,cafe,clear,"[0, 1, 1, 2, 0, 3, 1]"
39,POL_RULE_V2,REQ_0008,TRK_119,0.791,0.582,0.955,0.537,5,0,0,...,2025-01-04 10:05:00,카페에서 집중해서 공부 중이야,"[0,1,1,2,0,3,1]",joy,3,1.494175,studying,cafe,clear,"[0, 1, 1, 2, 0, 3, 1]"


In [42]:
group_train = df.loc[train_mask].groupby("request_id").size().to_list()
group_test = df.loc[test_mask].groupby("request_id").size().to_list()

In [56]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [43]:
from xgboost import XGBRanker

model = XGBRanker(
    objective="rank:ndcg",
    eval_metric="ndcg",
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42,
)

model.fit(
    X_train, y_train,
    group=group_train,
    eval_set=[(X_test, y_test)],
    eval_group=[group_test],
    verbose=False
)

,objective,'rank:ndcg'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.9
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'ndcg'


X      : track_features, mood_features, condition_features
y      : relevance (bucket)
group  : request_id
meta   : rank, track_id (학습 제외)

In [45]:
df_test["pred_score"] = model.predict(X_test)

topk = (
    df_test.sort_values(["request_id", "pred_score"], ascending=[True, False])
            .groupby("request_id")
            .head(5)[["request_id", "track_id", "pred_score", "relevance", "rank"]]
)

topk

,request_id,track_id,pred_score,relevance,rank
29,REQ_0006,TRK_083,0.513049,0,5
28,REQ_0006,TRK_046,0.252302,0,4
26,REQ_0006,TRK_117,-0.081247,2,2
25,REQ_0006,TRK_111,-0.482116,3,1
27,REQ_0006,TRK_120,-1.895979,1,3
36,REQ_0008,TRK_049,0.846891,2,2
35,REQ_0008,TRK_016,0.089860,3,1
37,REQ_0008,TRK_007,0.089860,1,3
38,REQ_0008,TRK_080,0.089860,0,4
39,REQ_0008,TRK_119,-0.734050,0,5


In [50]:
# 모델이 추천한 상위 K곡을 사용자에게 보여줬을 때, 요청 하나당 얻는 총 만족도의 평균
def mean_reward(df, order_col="pred_score", k=3, higher_is_better=True):
    asc = not higher_is_better
    return (
        df.sort_values(["request_id", order_col], ascending=[True, asc])
            .groupby("request_id")
            .head(k)
            .groupby("request_id")["relevance"]
            .sum()
            .mean()
    )

def metric(df, col, order_col="pred_score", k=3, higher_is_better=True):
    asc = not higher_is_better
    return (
        df.sort_values(["request_id", order_col], ascending=[True, asc])
            .groupby("request_id")
            .head(k)[col]
            .mean()
    )
    
for k in range(5,0,-1):
    print("K=", k,
          "(baseline)reward@", mean_reward(df_test, "rank", k),
          "(model)reward@", mean_reward(df_test, "pred_score", k),
          "click@", metric(df_test, "clicked", "pred_score", k), # 평균적으로 얼마나 클릭하는지
          "like@", metric(df_test, "liked", "pred_score", k) # 평균적으로 얼마나 좋아요 표시하는지
    )



K= 5 (baseline)reward@ 5.7 (model)reward@ 5.7 click@ 0.64 like@ 0.14
K= 4 (baseline)reward@ 3.3 (model)reward@ 5.4 click@ 0.725 like@ 0.175
K= 3 (baseline)reward@ 1.7 (model)reward@ 4.3 click@ 0.7333333333333333 like@ 0.16666666666666666
K= 2 (baseline)reward@ 0.7 (model)reward@ 2.7 click@ 0.7 like@ 0.2
K= 1 (baseline)reward@ 0.0 (model)reward@ 1.5 click@ 0.7 like@ 0.3


In [ ]:
def dcg_at_k(rels, k):
    # rels: 정렬된 relevance 리스트 (상위부터)
    